In [8]:
# main libraries
import numpy as np
import matplotlib.pyplot as plt
# torch library
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
# torchvision library
import torchvision
from torchvision import transforms, datasets
import TorchCommon as TC
import Network

In [2]:
#main pyperparametrs
valid_size=0.2
batch_size=64
epochs=50
lr=0.1
nrm_mean=0.5
nrm_std=0.5
num_workers=0

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# transform
trans=transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((nrm_mean,) , (nrm_std,))
])

In [4]:
#load dataset
train_data=datasets.MNIST(root='D:\GitHub\pytorch-lab\MLP\dataset',
                          train=True, transform=trans, download=True)

test_data=datasets.MNIST(root='D:\GitHub\pytorch-lab\MLP\dataset',
                          train=False, transform=trans, download=True)

100.0%


Extracting D:\GitHub\pytorch-lab\MLP\dataset\MNIST\raw\train-images-idx3-ubyte.gz to D:\GitHub\pytorch-lab\MLP\dataset\MNIST\raw



102.8%


Extracting D:\GitHub\pytorch-lab\MLP\dataset\MNIST\raw\train-labels-idx1-ubyte.gz to D:\GitHub\pytorch-lab\MLP\dataset\MNIST\raw



100.0%


Extracting D:\GitHub\pytorch-lab\MLP\dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to D:\GitHub\pytorch-lab\MLP\dataset\MNIST\raw



112.7%

Extracting D:\GitHub\pytorch-lab\MLP\dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to D:\GitHub\pytorch-lab\MLP\dataset\MNIST\raw



In [9]:
#sampler
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [10]:
#dataloader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)
valid_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=valid_sampler)
test_loader =DataLoader(test_data,  batch_size= batch_size, num_workers=num_workers, shuffle=True)

In [13]:
#create model and set loss function and optimizer
model=Network.Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

In [14]:
loss_valid_min=np.Inf
for epoch in range(1,epochs+1):
    loss_train, acc_train = TC.train(model,train_loader, device, optimizer, criterion, epoch)
    loss_valid, acc_valid = TC.valid(model, valid_loader, device, criterion)
    root="D:\GitHub\pytorch-lab\MLP\models"
    loss_valid_min = TC.save_model(model, optimizer, epoch, root, loss_valid_min, loss_valid)

############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 0.49	Acc_Train : 0.84
Loss_Valid: 0.25	Acc_Valid : 0.92

Save Model: YES
############################################################
###### Epoch 2 #############################################
############################################################
Loss_Train: 0.19	Acc_Train : 0.94
Loss_Valid: 0.15	Acc_Valid : 0.95

Save Model: YES
############################################################
###### Epoch 3 #############################################
############################################################
Loss_Train: 0.14	Acc_Train : 0.96
Loss_Valid: 0.14	Acc_Valid : 0.96

Save Model: YES
############################################################
###### Epoch 4 #############################################
############################################################
Loss_Train: 0

KeyboardInterrupt: 

In [15]:
model=Network.Net().to(device)
root='D:\GitHub\pytorch-lab\MLP\models\model_epoch_19_loss_valid_0.07.pt'
TC.test(model, test_loader, device, criterion, root)

############################################################
###### TEST INFO ###########################################
############################################################
Best Epoch : 19	Loss_Test : 0.07	Acc_Test : 0.98
############################################################
###### TEST INFO ###########################################
############################################################
